# Runnables 구조 검토

LCEL로 `runnable` 을 생성한 후에는 종종 이를 검사하여 어떤 일이 일어나고 있는지 더 잘 파악하고 싶을 것입니다.

이 노트북에서는 이를 수행하는 몇 가지 방법을 다룹니다.

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH13-LCEL-Advanced")

In [ ]:
# !pip install -qU faiss-cpu tiktoken

# 그래프를 그리기 위한 라이브러리 설치
# !pip install -qU grandalf

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    # 텍스트 데이터로부터 FAISS 벡터 저장소를 생성합니다.
    ["Teddy is an AI engineer who loves programming!"],
    embedding=OpenAIEmbeddings(),
)

# 벡터 저장소를 기반으로 retriever를 생성합니다.
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}  

Question: {question}"""

prompt = ChatPromptTemplate.from_template(
    template
)  # 템플릿을 기반으로 ChatPromptTemplate을 생성합니다.

model = ChatOpenAI(model="gpt-4o-mini")  # ChatOpenAI 모델을 초기화합니다.

# chain 을 생성합니다.
chain = (
    # 검색 컨텍스트와 질문을 지정합니다.
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt  # 프롬프트를 생성합니다.
    | model  # 언어 모델을 실행합니다.
    | StrOutputParser()  # 출력 결과를 문자열로 파싱합니다.
)

## 그래프 구성 확인

runnable의 그래프를 얻을 수 있습니다.


`chain.get_graph()` 메서드는 체인의 실행 그래프를 반환합니다.

- 이 메서드는 체인의 각 노드와 노드 간의 연결을 나타내는 그래프 객체를 반환합니다.
- 그래프의 노드는 체인의 각 단계를 나타내며, 에지(edge)는 단계 간의 데이터 흐름을 나타냅니다.


In [ ]:
# 체인의 그래프에서 노드를 가져옵니다.
chain.get_graph().nodes

In [ ]:
# 체인의 그래프에서 엣지를 가져옵니다.
chain.get_graph().edges

## 그래프 출력

그래프를 출력하면 이해하기 쉬운 형태로 표현할 수 있습니다.

비록 출력 결과가 매우 읽기 쉽지는 않지만, 출력을 통해 보다 이해하기 쉬운 형태로 그래프를 확인할 수 있습니다.


In [ ]:
# 체인의 그래프를 ASCII 형식으로 출력합니다.
chain.get_graph().print_ascii()

## 프롬프트 가져오기

체인에서 중요한 부분은 사용되는 프롬프트입니다.


`chain.get_prompts()` 메서드는 체인에서 사용되는 프롬프트(prompt) 객체의 리스트를 반환합니다.


In [ ]:
chain.get_prompts()  # 체인에서 사용되는 프롬프트를 가져옵니다.